In [1]:
import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import matplotlib
import time
import copy
from datetime import datetime
import shutil
from tqdm import tqdm

from torchvision import transforms, models

# rs = 0
# random.seed(rs)
# np.random.seed(rs)
# torch.manual_seed(rs)
# torch.cuda.manual_seed(rs)
# torch.backends.cudnn.deterministic = True

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['cleaned-vs-dirty-v2-rembg', 'platesv2']


In [3]:
#hyperparameters
images_after_rembg = True
with_validation = True
dropout_p = 0.3

model = models.resnet152(pretrained=True) #загружаем предтренированную на imagenet1000 модель
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

optimizer_type = 'SGD' #Adam or SGD

optimizer_weight_decay = 1e-5 #[0, 1e-3, 1e-4, 1e-5] for Adam and SGD

#Adam optimyzer
optimizer_Adam_lr = 1.0e-2
optimizer_Adam_amsgrad = False

#SGD optimyzer
optimizer_SGD_lr = 0.1
optimizer_SGD_momentum = 0.5 #[0 - 1]

scheduler_step_size = 10
scheduler_gamma = 0.9


num_epochs = 100
batch_size = 10
batch_val_size = 744
class_names = ['cleaned', 'dirty']

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

In [4]:
train_transforms_rot = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=(0, 180)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_transforms_rot_autoaug = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=(0, 180)),
    transforms.AutoAugment(),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_transforms_rot_cjitter = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=(0, 180)),
    transforms.ColorJitter(0.2, 0.3, 0.3, 0.2),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
# #extract original files
# import zipfile
# with zipfile.ZipFile('../input/platesv2/plates.zip', 'r') as zip_obj:
#    # Extract all the contents of zip file in current directory
#    zip_obj.extractall('/kaggle/working/')

In [6]:
#copy files after remove background (by rembg, https://github.com/danielgatis/rembg)
!cp -R ../input/cleaned-vs-dirty-v2-rembg/platesv2/platesv2_rbg ./plates_rbg
#copy original files
!cp -R ../input/cleaned-vs-dirty-v2-rembg/platesv2/platesv2 ./plates

#создаем в test поддиректорию не существующего класса unknown, и перемещаем туда test изображения
#т.к. .ImageFolder требует, что` бы в целевой директории обязательно были поддиректории=классы
#shutil.movetree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))
!mkdir ./plates_rbg/test/unknown/
!mkdir ./plates/test/unknown
!mv -f ./plates_rbg/test/*.png ./plates_rbg/test/unknown
!mv -f ./plates/test/*.jpg ./plates/test/unknown

In [7]:
print('After zip extraction / copying:')
print(os.listdir("/kaggle/working/"))

After zip extraction / copying:
['plates', 'plates_rbg', '__notebook__.ipynb']


In [8]:
# if images_after_rembg:
#     data_root = '/kaggle/working/plates_rbg/'
# else:
#     data_root = '/kaggle/working/plates/'
# print(os.listdir(data_root))

In [9]:
# #remove all files .DS_Store
# for parent, dirnames, filenames in os.walk(data_root):
#     for fn in filenames:
#         if fn.lower().endswith('.ds_store'):
#             os.remove(os.path.join(parent, fn))

In [10]:
# train_dir = data_root + 'train'
# val_dir = data_root + 'val'
# test_dir = data_root + 'test'

In [11]:
# # The shutil module offers a number of high-level operations on files and collections of files. 
# # In particular, functions are provided which support file copying and removal.
# import shutil
# from tqdm import tqdm

# train_dir = 'train'
# val_dir = 'val'

# class_names = ['cleaned', 'dirty']

# #create dirs ./train/cleaned, /train/dirty, ./val/cleaned, /val/dirty,
# for dir_name in [train_dir, val_dir]:
#     for class_name in class_names:
#         os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

# #fill directories train (5/6)=83% and val (1/6)=17%, 
# for class_name in class_names:
#     source_dir = os.path.join(data_root, 'train', class_name)
#     for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
#         if (i % 6 == 0) and with_validation:
#             dest_dir = os.path.join(val_dir, class_name)
#         else:
#             dest_dir = os.path.join(train_dir, class_name)
#         shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

In [12]:
# # Transforms are common image transformations. They can be chained together using Compose. 
# # Most transform classes have a function equivalent: functional transforms give fine-grained 
# # control over the transformations. This is useful if you have to build a more complex transformation 
# # pipeline (e.g. in the case of segmentation tasks).

# # ILLUSTRATION OF TRANSFORMS: https://pytorch.org/vision/stable/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py

# train_transforms = transforms.Compose([
#     #!!очень спорное применение RandomResizedCrop для нашего случая, 
#     #т.к. по итогу абсолютно получается рандомно вырезанный, сжатый/растянутый кусок исходного изображения
#     #transforms.RandomResizedCrop(224),
#     transforms.RandomHorizontalFlip(), #с вероятностью 50% отражаем по горизонтали
#     transforms.RandomVerticalFlip(),
#     transforms.RandomRotation(degrees=(0, 180)),
#     #transforms.AutoAugment(),
#     #transforms.ColorJitter(0.2, 0.3, 0.3, 0.2),
#     #transforms.Resize((224, 224)),
#     #transforms.RandomResizedCrop(224),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     #нормализуем в соостветсвии с тем, как, были предобработаны изображения imagenet1000, при обучении resnet
#     # mean = [0.485, 0.456, 0.406], эти значения вычитаются из RGB каналов изображения (т.е. нормализуем смещение)
#     # std = [0.229, 0.224, 0.225], на эти значения делим RGB каналы изображения 
#     #      (т.е. нормализцем среднеквадратичное (стандартное) отклонение, std^2 = var)
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])

# val_transforms = transforms.Compose([
#     #transforms.Resize((224, 224)), #важно, не квадратные изображения будут сжиматься/растягиваться!
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])


# #получаем объект типа .ImageFolder, для дальнейшего использования через torch.utils.data.DataLoader
# #по сути связывает директорию с изображениями, разбитыми ко поддиректориям=классам, с трансформациями, 
# #которые будут производиться при дальнейшей загрузке изображений в память (в том числе в композиции 
# #трансформаций выше сразу зашито преобразование в тензор)
# #Важные особенности:
# # - указанная директория обязательно должна содержать поддиректории соответсвующие названиям классов, 
# #   уже в свою очередь в которых должны находиться изображения
# # - по итогу, мы получаем объект типа ImageFolder, который на первый взгляд является последовательностью 
# #   (sequential), длины = кол-ву изображений во всех поддиректориях, но по факту, является только ссылками
# #   на изображения, т.е. в память сразу они не загружаются, а только в момент обращения к конкретному
# #   объекту; простая проверка, если удалить изображение(-я) с диска, заранее созданный объект ImageFolder
# #   уже не сможет предоставить доступ к соответсвующему удаленному изображению объекту (с исключением о 
# #   невозможности найти файл)
# # - важно понимать!, что каждый раз при обращении к конкретному элементу (например с индексом 0), весь 
# #   процесс загрузки с диска и применение трансформаций будет запускаться заново, как следствие, если мы
# #   имеем рандомные трансформации, т.е. каждый раз при обращении к одному и тому же объекту мы будем 
# #   получать разные результаты
# # - для нашего случая трансформаций (где присутсвует .ToTensor()), мы получим последовательность кортежей, 
# #   из 2-х подэлементов, 0-й элемент = тензору соответсвующему изображению, 1-й элемент целое число, 
# #   отнесение к классу (насколько я понял, классы назначаются в порядке алфавитной сортировки поддиректорий, 
# #   т.е. для нашего случая cleaned = 0, dirty = 1)

# train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)
# if with_validation:
#     val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)
# len(train_dataset)
# if with_validation:
#     len(val_dataset)

In [13]:
# # На основе .ImageFolder генерируем подобие итератора/последовательности псевдо-батчей указанного размера 
# # (в зависимости от shuffle, перемешиваем их или нет), псевдо-, потому что данные не загружаются сразу, 
# # а только в момет обращения к ним, для этого указывается количество воркеров (для многопоточности).
# # Важные примечания:
# # - с учетом нашей реализации .ImageFolder, где происходят рандомные трансформации, каждый раз получем
# #   рандомно измененные изображения, даже для одинаковых батчей
# # - изображения в каждом батче, так же перемешиваются каждый раз при вызове запросе батча
# # - это все же, как понял это некоторый микс итератора/последовательности, т.к. например, у объекта есть 
# #   размер len(dataloader), но при этом обратиться к индексу нельзя, просто вызвать next(dataloader)
# #   нельзя, зато можно вызвать предварительно приведя к классическому итератору через next(iter(dataloader))
# # - при получении следующего батча (пример выше), возвращает 2 объекта/датасета с фичами (X) и labels (y)
# #   (для нашего случая возвращает 2 тензора, каждый батч фичей будет torch.Size([8, 3, 224, 224]), батч 
# #   лейблов torch.Size([8]))

# train_dataloader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
# if with_validation:
#     val_dataloader = torch.utils.data.DataLoader(
#         val_dataset, batch_size=batch_val_size, shuffle=False, num_workers=2)

In [14]:
# print(len(train_dataloader))
# if with_validation:
#     print(len(val_dataloader))

In [15]:
# X_batch, y_batch = next(iter(train_dataloader))
# mean = np.array([0.485, 0.456, 0.406])
# std = np.array([0.229, 0.224, 0.225])
# #permute(1, 2, 0) необходим, т.к. для нескольких каналов, plt принимает тензор, 
# #где измерение (dim) каналов на последнем месте, а в торче каналы стоят перед 
# #размерами изображения (на 2 месте)
# #так же производим денормализацию (там делили вычитали, здесь умножаем складываем), 
# #противоположную той, которую произвели в трансформере во время загрузки изображения
# plt.imshow(X_batch[0].permute(1, 2, 0).numpy() * std + mean); 

In [16]:
def show_input(input_tensor, title='', 
               mean = np.array([0.485, 0.456, 0.406]), 
               std = np.array([0.229, 0.224, 0.225])):
    '''Функция реализующая вывод изображений с заголовком = классу'''
    
    #permute(1, 2, 0) необходим, т.к. для нескольких каналов, plt принимает тензор, 
    #где измерение (dim) каналов на последнем месте, а в торче каналы стоят перед 
    #размерами изображения (на 2 месте)
    #так же производим денормализацию (там делили вычитали, здесь умножаем складываем), 
    #противоположную той, которую произвели в трансформере во время загрузки изображения
    image = input_tensor.permute(1, 2, 0).numpy()
    image = std * image + mean
    plt.imshow(image.clip(0, 1)) #.clip загоняет все значения в заданный интервал, т.е. [0, 1]
    plt.title(title)
    plt.show()
    plt.pause(0.001)

def show_inputs_from_loader(dataloader, 
               mean = np.array([0.485, 0.456, 0.406]), 
               std = np.array([0.229, 0.224, 0.225])):
    
    X_batch, y_batch = next(iter(dataloader))

    for x_item, y_item in zip(X_batch, y_batch):
        show_input(x_item, title=class_names[y_item], mean=mean, std=std)

In [17]:
# show_inputs_from_loader(train_dataloader)

In [18]:
# if with_validation:
#     iter_val_dataloader = iter(val_dataloader)
#     inputs_slice = - batch_size

In [19]:
# if with_validation:
#     inputs_slice += batch_size
#     inputs, labels = next(iter_val_dataloader)

# for x_item, y_item in zip(inputs, labels):
#     show_input(x_item, title=class_names[y_item])

In [20]:
def show_experiments_plots(accuracies, losses, figsize = (16.0, 6.0), with_validation = True):
    matplotlib.rcParams['figure.figsize'] = figsize
    
    for experiment_id in accuracies.keys():
        print('{:-<100}'.format(experiment_id))
        epoch_max_acc = np.array(accuracies[experiment_id]['val']).argmax()
        print('Max val accuracy on: Epoch = {:>3},     ACCURACY: val = {:.3f}, train = {:.3f},     LOSS: val = {:.3f}, train = {:.3f}'\
              .format(epoch_max_acc, 
                      accuracies[experiment_id]['val'][epoch_max_acc], 
                      accuracies[experiment_id]['train'][epoch_max_acc],
                      losses[experiment_id]['val'][epoch_max_acc],
                      losses[experiment_id]['train'][epoch_max_acc]))
        epoch_min_loss = np.array(losses[experiment_id]['val']).argmin()
        print('Min val loss on:     Epoch = {:>3},     ACCURACY: val = {:.3f}, train = {:.3f},     LOSS: val = {:.3f}, train = {:.3f}'\
              .format(epoch_min_loss, 
                      accuracies[experiment_id]['val'][epoch_min_loss], 
                      accuracies[experiment_id]['train'][epoch_min_loss],
                      losses[experiment_id]['val'][epoch_min_loss],
                      losses[experiment_id]['train'][epoch_min_loss]))
    
    for experiment_id in accuracies.keys():
        plt.plot(accuracies[experiment_id]['val'], label=experiment_id + ' val')
    plt.legend()
    plt.title('Validation Accuracy (Val only)')
    plt.show()

    for experiment_id in accuracies.keys():
        plt.plot(accuracies[experiment_id]['val'], label=experiment_id + ' val')
        plt.plot(accuracies[experiment_id]['train'], label=experiment_id + ' train')
    plt.legend()
    plt.title('Validation Accuracy (Val/Train)');
    plt.show()

    for experiment_id in losses.keys():
        plt.plot(losses[experiment_id]['val'], label=experiment_id  + ' val')
    plt.legend()
    plt.title('Validation Loss (Val only)');
    plt.show()

    for experiment_id in losses.keys():
        plt.plot(losses[experiment_id]['val'], label=experiment_id  + ' val')
        plt.plot(losses[experiment_id]['train'], label=experiment_id  + ' train')
    plt.legend()
    plt.title('Validation Loss (Val/Train)');
    plt.show()

In [21]:
# val_inputs, val_labels = next(iter(val_dataloader))
# len(val_inputs)

In [22]:
# val_inputs = val_inputs.to(device)
# val_labels = val_labels.to(device)

In [23]:
def train_model(model, loss, optimizer, scheduler, num_epochs, train_dataloader, val_inputs, val_labels,
                device = 'cpu', with_validation = True, experiment_name = 'default_params'):
    
    print("############## Start experiment with name: {} ##############".format(experiment_name))
    accuracy_history = {}
    loss_history = {}
    accuracy_history['train'] = []
    loss_history['train'] = []
    accuracy_history['val'] = []
    loss_history['val'] = []
    
    if with_validation:
        best_acc_type = 'val'
    else:
        best_acc_type = 'train'
    EMA_momentum = 0.7
    best_accuracy = 0
    best_EMA_accuracy = 0
    best_accuracy_epoch = 0
    best_EMA_accuracy_epoch = 0
    current_EMA_accuracy = 0
    
    for epoch in range(num_epochs):
        
        #####train phase######
        train_accuracy_epoch = [] #for statistics
        train_loss_epoch = [] #for statistics
        
        model.train()  # Set model to training mode

        for inputs, labels in train_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                preds = model(inputs) #аналогично вызову метода model.forward(inputs)
                loss_value = loss(preds, labels)
                
                loss_value.backward()
                optimizer.step()
                scheduler.step() #no grad step, only change lr of optimizer (if Adam)
                
                train_accuracy_epoch.append(float((preds.argmax(dim=1) == labels.data).float().mean().data))
                #.item() Returns the value of this tensor as a standard Python number. This only works for tensors with one element.
                train_loss_epoch.append(loss_value.item())
                
        #####validation phase######
        if with_validation:
            val_accuracy_epoch = []
            val_loss_epoch = []

            model.eval()   #Set model to evaluate mode

#             for inputs, labels in val_dataloader:
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)
            with torch.no_grad(): #не храним градиенты для теста, многоркатно сокращает потребление памяти                  
                preds = model(val_inputs) #аналогично вызову метода model.forward(inputs)
                loss_value = loss(preds, val_labels)

                val_accuracy_epoch.append(float((preds.argmax(dim=1) == val_labels.data).float().mean().data))
                val_loss_epoch.append(loss_value.item())
                    
        ########ending of epoch#########
        if with_validation:
            accuracy_history['val'].append(sum(val_accuracy_epoch) / len(val_accuracy_epoch))
            loss_history['val'].append(sum(val_loss_epoch) / len(val_loss_epoch))
        else:
            accuracy_history['val'].append(0)
            loss_history['val'].append(0)
            
        accuracy_history['train'].append(sum(train_accuracy_epoch) / len(train_accuracy_epoch))
        loss_history['train'].append(sum(train_loss_epoch) / len(train_loss_epoch))    
        
        #save best model copy (type by 'train' or 'val')
        #!!TASK!! Возможно нужна не просто модель по одному всплеску, а лучшая по среднему или ЕМА 
        #за несколько соседних экспериментов
        if accuracy_history[best_acc_type][-1] > best_accuracy:
            best_accuracy = accuracy_history[best_acc_type][-1]
            best_accuracy_model = copy.deepcopy(model)
            best_accuracy_epoch = epoch
        if epoch > 0:
            current_EMA_accuracy = (1 - EMA_momentum) * current_EMA_accuracy + EMA_momentum * accuracy_history[best_acc_type][-1]
            if current_EMA_accuracy > best_EMA_accuracy:
                best_EMA_accuracy = current_EMA_accuracy
                best_EMA_accuracy_epoch = epoch
                best_EMA_accuracy_model = copy.deepcopy(model)
        else:
            current_EMA_accuracy = accuracy_history[best_acc_type][-1]
            
        print('Epoch = {:>3}/{},     ACCURACY: val = {:.3f}, train = {:.3f}, ema = {:.3f},      LOSS: val = {:.3f}, train = {:.3f}'\
                  .format(epoch,
                          num_epochs - 1,
                          accuracy_history['val'][-1], 
                          accuracy_history['train'][-1],
                          current_EMA_accuracy,
                          loss_history['val'][-1],
                          loss_history['train'][-1]), 
                  flush=True)
    print("Best accuracy = {:.3f}, on epoch = {}, Best EMA accuracy = {:.3f}, on epoch = {}".format(best_accuracy,
                                                                                                    best_accuracy_epoch, 
                                                                                                    best_EMA_accuracy,
                                                                                                    best_EMA_accuracy_epoch))
    
    print("************** End experiment with name: {} **************".format(experiment_name))
    print(' ')
    return best_accuracy_model, best_EMA_accuracy_model, best_accuracy, best_EMA_accuracy, accuracy_history, loss_history

In [24]:
def run_experiment(experiment_name = 'default_params',
                  images_after_rembg = True,
                  with_validation = True,
                  optimizer_type = 'Adam',
                  dropout_p = 0,
                  optimizer_weight_decay = 0,
                  optimizer_Adam_lr = 1.0e-2,
                  optimizer_Adam_amsgrad = False,
                  optimizer_SGD_lr = 0.1,
                  optimizer_SGD_momentum = 0.9,
                  scheduler_step_size = 8,
                  scheduler_gamma = 0.8,
                  num_epochs = 100,
                  batch_size = 10,
                  batch_val_size = 744,
                  train_transforms = train_transforms_rot,
                  val_transforms = val_transforms,
                  class_names = ['cleaned', 'dirty'],
                  device = device):
    if images_after_rembg:
        data_root = '/kaggle/working/plates_rbg/'
    else:
        data_root = '/kaggle/working/plates/'
    train_dir = data_root + 'train'
    val_dir = data_root + 'val'
    
    train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)
    if with_validation:
        val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)
    
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    if with_validation:
        val_dataloader = torch.utils.data.DataLoader(
            val_dataset, batch_size=batch_val_size, shuffle=False, num_workers=2)
    val_inputs, val_labels = next(iter(val_dataloader))
    val_inputs = val_inputs.to(device)
    val_labels = val_labels.to(device)
    
    model = models.resnet152(pretrained=True) #загружаем предтренированную на imagenet1000 модель
    
    # Disable grad for all conv layers (скорей всего морозим градиенты не только conv)
    for param in model.parameters():
        param.requires_grad = False

    if dropout_p > 0:
        model.fc = torch.nn.Sequential(torch.nn.Dropout(dropout_p), torch.nn.Linear(model.fc.in_features, 2))
    else:
        model.fc = torch.nn.Linear(model.fc.in_features, 2)

    model = model.to(device)

    loss = torch.nn.CrossEntropyLoss()

    if optimizer_type == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=optimizer_Adam_lr, weight_decay=optimizer_weight_decay, amsgrad = optimizer_Adam_amsgrad)

    elif optimizer_type == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=optimizer_SGD_lr, momentum=optimizer_SGD_momentum, weight_decay=optimizer_weight_decay)

    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step_size, gamma=scheduler_gamma)
    
    best_accuracy_model, best_EMA_accuracy_model, best_accuracy, best_EMA_accuracy, accuracy_history, loss_history = \
    train_model(model, loss, optimizer, scheduler, train_dataloader=train_dataloader, val_inputs=val_inputs, val_labels=val_labels,
                device = device, num_epochs=num_epochs, with_validation = with_validation, experiment_name = experiment_name)
    
    return best_accuracy_model, best_EMA_accuracy_model, best_accuracy, best_EMA_accuracy, accuracy_history, loss_history

In [25]:
#создаем класс ImageFolderWithPaths, на основе ImageFolder, с единственным изменением метода .__getitem__
#что бы кортеж элементов, помимо изображения и класса, содержал еще 3-й элемент = полный путь к изображению
#это необходимо для формирования submit csv, где первый столбец = индекcу фото (имя файла без расширения),
#второй столбец = отнесению к классу
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,)) #сумма кортежей = concat из всех элементов
        return tuple_with_path

In [26]:
def get_test_predictions(model, images_after_rembg = True, batch_size = 10):
    if images_after_rembg:
        data_root = '/kaggle/working/plates_rbg/'
    else:
        data_root = '/kaggle/working/plates/'
    test_dir = data_root + 'test'
    test_dataset = ImageFolderWithPaths(test_dir, val_transforms)
    test_dataloader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    model.eval() #не забываем перевести модель в eval режим

    test_predictions = []
    test_img_paths = []
    for inputs, labels, paths in iter(test_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.no_grad(): #не храним градиенты для теста, многоркатно сокращает потребление памяти
            preds = model(inputs)
            test_predictions.append(
                torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
        test_img_paths.extend(paths)

    test_predictions = np.concatenate(test_predictions) #преобразуем list в np.array
    
    predictions_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})
    predictions_df['id'] = predictions_df['id'].str.replace(test_dir + '/unknown/', '', regex=False)
    predictions_df['id'] = predictions_df['id'].str.replace('.jpg', '', regex=False)
    predictions_df['id'] = predictions_df['id'].str.replace('.png', '', regex=False)
    predictions_df.set_index('id', inplace=True)
    
    return predictions_df

In [27]:
# accuracies = {}
# losses = {}

In [28]:
# experiment_name = 'resnet152_test_run_experiment'
# best_accuracy_model, best_EMA_accuracy_model, best_accuracy, best_EMA_accuracy, accuracies[experiment_name], losses[experiment_name] = \
#    run_experiment(num_epochs=5)

In [29]:
# submission_df = get_test_predictions(best_accuracy_model, images_after_rembg = images_after_rembg, batch_size = batch_size)
# submission_df

In [30]:
list_of_experiments_params = [
#     {'experiment_name':'Adam_default_params', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'Adam',
#      'optimizer_Adam_lr': 1.0e-2, 'optimizer_Adam_amsgrad': False, 'dropout_p': 0, 'optimizer_weight_decay': 0, 
#      'scheduler_step_size': 8, 'scheduler_gamma': 0.8, 'num_epochs': 100, 'batch_size': 40, 'train_transforms': train_transforms_rot}, 
 
#     {'experiment_name':'Adam_default_params_autoaug', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'Adam',
#      'optimizer_Adam_lr': 1.0e-2, 'optimizer_Adam_amsgrad': False, 'dropout_p': 0, 'optimizer_weight_decay': 0, 
#      'scheduler_step_size': 8, 'scheduler_gamma': 0.8, 'num_epochs': 100, 'batch_size': 40, 'train_transforms': train_transforms_rot_autoaug},
    
#     {'experiment_name':'Adam_default_params_cjitter', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'Adam',
#      'optimizer_Adam_lr': 1.0e-2, 'optimizer_Adam_amsgrad': False, 'dropout_p': 0, 'optimizer_weight_decay': 0, 
#      'scheduler_step_size': 8, 'scheduler_gamma': 0.8, 'num_epochs': 100, 'batch_size': 40, 'train_transforms': train_transforms_rot_cjitter},
 
    
#     {'experiment_name':'Adam_default_params_no_rembg', 'images_after_rembg': False, 'with_validation': True, 'optimizer_type': 'Adam',
#      'optimizer_Adam_lr': 1.0e-2, 'optimizer_Adam_amsgrad': False, 'dropout_p': 0, 'optimizer_weight_decay': 0, 
#      'scheduler_step_size': 8, 'scheduler_gamma': 0.8, 'num_epochs': 100, 'batch_size': 40, 'train_transforms': train_transforms_rot}, 
 
#     {'experiment_name':'Adam_default_params_autoaug_no_rembg', 'images_after_rembg': False, 'with_validation': True, 'optimizer_type': 'Adam',
#      'optimizer_Adam_lr': 1.0e-2, 'optimizer_Adam_amsgrad': False, 'dropout_p': 0, 'optimizer_weight_decay': 0, 
#      'scheduler_step_size': 8, 'scheduler_gamma': 0.8, 'num_epochs': 100, 'batch_size': 40, 'train_transforms': train_transforms_rot_autoaug},
    
#     {'experiment_name':'Adam_default_params_cjitter_no_rembg', 'images_after_rembg': False, 'with_validation': True, 'optimizer_type': 'Adam',
#      'optimizer_Adam_lr': 1.0e-2, 'optimizer_Adam_amsgrad': False, 'dropout_p': 0, 'optimizer_weight_decay': 0, 
#      'scheduler_step_size': 8, 'scheduler_gamma': 0.8, 'num_epochs': 100, 'batch_size': 40, 'train_transforms': train_transforms_rot_cjitter},

    
    {'experiment_name':'Adam_default_params_no_rembg_drop_0.4', 'images_after_rembg': False, 'with_validation': True, 'optimizer_type': 'Adam',
     'optimizer_Adam_lr': 1.0e-2, 'optimizer_Adam_amsgrad': False, 'dropout_p': 0.4, 'optimizer_weight_decay': 0, 
     'scheduler_step_size': 8, 'scheduler_gamma': 0.8, 'num_epochs': 30, 'batch_size': 40, 'train_transforms': train_transforms_rot}, 
 
    {'experiment_name':'Adam_default_params_autoaug_no_rembg_drop_0.4', 'images_after_rembg': False, 'with_validation': True, 'optimizer_type': 'Adam',
     'optimizer_Adam_lr': 1.0e-2, 'optimizer_Adam_amsgrad': False, 'dropout_p': 0.4, 'optimizer_weight_decay': 0, 
     'scheduler_step_size': 8, 'scheduler_gamma': 0.8, 'num_epochs': 30, 'batch_size': 40, 'train_transforms': train_transforms_rot_autoaug},
    
    {'experiment_name':'Adam_default_params_cjitter_no_rembg_drop_0.4', 'images_after_rembg': False, 'with_validation': True, 'optimizer_type': 'Adam',
     'optimizer_Adam_lr': 1.0e-2, 'optimizer_Adam_amsgrad': False, 'dropout_p': 0.4, 'optimizer_weight_decay': 0, 
     'scheduler_step_size': 8, 'scheduler_gamma': 0.8, 'num_epochs': 30, 'batch_size': 40, 'train_transforms': train_transforms_rot_cjitter},
    
    
    {'experiment_name':'Adam_default_params_drop_0.6', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'Adam',
     'optimizer_Adam_lr': 1.0e-2, 'optimizer_Adam_amsgrad': False, 'dropout_p': 0.6, 'optimizer_weight_decay': 0, 
     'scheduler_step_size': 8, 'scheduler_gamma': 0.8, 'num_epochs': 30, 'batch_size': 40, 'train_transforms': train_transforms_rot}, 
 
    {'experiment_name':'Adam_default_params_autoaug_drop_0.6', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'Adam',
     'optimizer_Adam_lr': 1.0e-2, 'optimizer_Adam_amsgrad': False, 'dropout_p': 0.6, 'optimizer_weight_decay': 0, 
     'scheduler_step_size': 8, 'scheduler_gamma': 0.8, 'num_epochs': 30, 'batch_size': 40, 'train_transforms': train_transforms_rot_autoaug},
    
    {'experiment_name':'Adam_default_params_cjitter_drop_0.6', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'Adam',
     'optimizer_Adam_lr': 1.0e-2, 'optimizer_Adam_amsgrad': False, 'dropout_p': 0.6, 'optimizer_weight_decay': 0, 
     'scheduler_step_size': 8, 'scheduler_gamma': 0.8, 'num_epochs': 30, 'batch_size': 40, 'train_transforms': train_transforms_rot_cjitter},    
    
    
    
    
    
#     {'experiment_name':'SGD_default_params', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'SGD',
#      'optimizer_SGD_lr': 0.1, 'optimizer_SGD_momentum': 0.9, 'dropout_p': 0, 'optimizer_weight_decay': 0, 
#      'scheduler_step_size': 8,'scheduler_gamma': 0.8, 'num_epochs': 100, 'batch_size': 40, 'train_transforms': train_transforms_rot},
    
#     {'experiment_name':'SGD_default_params_autoaug', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'SGD',
#      'optimizer_SGD_lr': 0.1, 'optimizer_SGD_momentum': 0.9, 'dropout_p': 0, 'optimizer_weight_decay': 0, 
#      'scheduler_step_size': 8,'scheduler_gamma': 0.8, 'num_epochs': 100, 'batch_size': 40, 'train_transforms': train_transforms_rot_autoaug},

#     {'experiment_name':'SGD_default_params_cjitter', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'SGD',
#      'optimizer_SGD_lr': 0.1, 'optimizer_SGD_momentum': 0.9, 'dropout_p': 0, 'optimizer_weight_decay': 0, 
#      'scheduler_step_size': 8,'scheduler_gamma': 0.8, 'num_epochs': 100, 'batch_size': 40, 'train_transforms': train_transforms_rot_cjitter},

    
#     {'experiment_name':'SGD_default_params_no_rembg', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'SGD',
#      'optimizer_SGD_lr': 0.1, 'optimizer_SGD_momentum': 0.9, 'dropout_p': 0, 'optimizer_weight_decay': 0, 
#      'scheduler_step_size': 8,'scheduler_gamma': 0.8, 'num_epochs': 100, 'batch_size': 40, 'train_transforms': train_transforms_rot},
    
#     {'experiment_name':'SGD_default_params_autoaug_no_rembg', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'SGD',
#      'optimizer_SGD_lr': 0.1, 'optimizer_SGD_momentum': 0.9, 'dropout_p': 0, 'optimizer_weight_decay': 0, 
#      'scheduler_step_size': 8,'scheduler_gamma': 0.8, 'num_epochs': 100, 'batch_size': 40, 'train_transforms': train_transforms_rot_autoaug},

#     {'experiment_name':'SGD_default_params_cjitter_no_rembg', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'SGD',
#      'optimizer_SGD_lr': 0.1, 'optimizer_SGD_momentum': 0.9, 'dropout_p': 0, 'optimizer_weight_decay': 0, 
#      'scheduler_step_size': 8,'scheduler_gamma': 0.8, 'num_epochs': 100, 'batch_size': 40, 'train_transforms': train_transforms_rot_cjitter},
    
    
#     {'experiment_name':'SGD_default_params_drop_0.2', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'SGD',
#      'optimizer_SGD_lr': 0.1, 'optimizer_SGD_momentum': 0.9, 'dropout_p': 0.2, 'optimizer_weight_decay': 0, 
#      'scheduler_step_size': 8,'scheduler_gamma': 0.8, 'num_epochs': 100, 'batch_size': 40, 'train_transforms': train_transforms_rot},
    
#     {'experiment_name':'SGD_default_params_autoaug_drop_0.2', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'SGD',
#      'optimizer_SGD_lr': 0.1, 'optimizer_SGD_momentum': 0.9, 'dropout_p': 0.2, 'optimizer_weight_decay': 0, 
#      'scheduler_step_size': 8,'scheduler_gamma': 0.8, 'num_epochs': 100, 'batch_size': 40, 'train_transforms': train_transforms_rot_autoaug},

#     {'experiment_name':'SGD_default_params_cjitter_drop_0.2', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'SGD',
#      'optimizer_SGD_lr': 0.1, 'optimizer_SGD_momentum': 0.9, 'dropout_p': 0.2, 'optimizer_weight_decay': 0, 
#      'scheduler_step_size': 8,'scheduler_gamma': 0.8, 'num_epochs': 100, 'batch_size': 40, 'train_transforms': train_transforms_rot_cjitter},
    
    
    {'experiment_name':'SGD_default_params_lr_0.01_mom_0.9_drop_0.6', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'SGD',
     'optimizer_SGD_lr': 0.01, 'optimizer_SGD_momentum': 0.9, 'dropout_p': 0.6, 'optimizer_weight_decay': 0, 
     'scheduler_step_size': 8,'scheduler_gamma': 0.8, 'num_epochs': 30, 'batch_size': 40, 'train_transforms': train_transforms_rot},
    
    {'experiment_name':'SGD_default_params_lr_0.01_mom_0.9_autoaug_drop_0.6', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'SGD',
     'optimizer_SGD_lr': 0.01, 'optimizer_SGD_momentum': 0.9, 'dropout_p': 0.6, 'optimizer_weight_decay': 0, 
     'scheduler_step_size': 8,'scheduler_gamma': 0.8, 'num_epochs': 30, 'batch_size': 40, 'train_transforms': train_transforms_rot_autoaug},

    {'experiment_name':'SGD_default_params_lr_0.01_mom_0.9_cjitter_drop_0.6', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'SGD',
     'optimizer_SGD_lr': 0.01, 'optimizer_SGD_momentum': 0.9, 'dropout_p': 0.6, 'optimizer_weight_decay': 0, 
     'scheduler_step_size': 8,'scheduler_gamma': 0.8, 'num_epochs': 30, 'batch_size': 40, 'train_transforms': train_transforms_rot_cjitter},
    
    
    
    {'experiment_name':'SGD_default_params_lr_0.3_mom_0.6_drop_0.5', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'SGD',
     'optimizer_SGD_lr': 0.3, 'optimizer_SGD_momentum': 0.6, 'dropout_p': 0.5, 'optimizer_weight_decay': 0, 
     'scheduler_step_size': 8,'scheduler_gamma': 0.8, 'num_epochs': 30, 'batch_size': 40, 'train_transforms': train_transforms_rot},
    
    {'experiment_name':'SGD_default_params_lr_0.3_mom_0.6_autoaug_drop_0.5', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'SGD',
     'optimizer_SGD_lr': 0.3, 'optimizer_SGD_momentum': 0.6, 'dropout_p': 0.5, 'optimizer_weight_decay': 0, 
     'scheduler_step_size': 8,'scheduler_gamma': 0.8, 'num_epochs': 30, 'batch_size': 40, 'train_transforms': train_transforms_rot_autoaug},

    {'experiment_name':'SGD_default_params_lr_0.3_mom_0.6_cjitter_drop_0.5', 'images_after_rembg': True, 'with_validation': True, 'optimizer_type': 'SGD',
     'optimizer_SGD_lr': 0.3, 'optimizer_SGD_momentum': 0.6, 'dropout_p': 0.5, 'optimizer_weight_decay': 0, 
     'scheduler_step_size': 8,'scheduler_gamma': 0.8, 'num_epochs': 30, 'batch_size': 40, 'train_transforms': train_transforms_rot_cjitter}
]

In [31]:
def run_all_experiments(list_of_experiments_params = [], repeat_count = 6, quantile_threshold = 1.0 - 486/744):
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    batch_val_size = 744
    output_of_experiments = []
    global_best_accuracy = 0
    global_best_EMA_accuracy = 0
    
    #итерируемся по заданным типам экспериментов
    for (experiment_index, experiment_params) in enumerate(list_of_experiments_params):
        output_of_experiments.append({'experiment_name': experiment_params['experiment_name']})
        best_accuracy = 0
        best_EMA_accuracy = 0
        #проводим repeat_count каждого типа эксперимента, сохраняем только лучший из 3-х вариант
        for i in range(repeat_count):
            curr_accuracy_model, curr_EMA_accuracy_model, \
            curr_accuracy, curr_EMA_accuracy, \
            curr_accuracies_history, curr_losses_history = run_experiment(
                                                    **experiment_params, 
                                                    batch_val_size = batch_val_size,
                                                    device = device)
            #check/save best of repeats results
            if curr_accuracy > best_accuracy:
                best_accuracy_model = curr_accuracy_model
                best_accuracy = curr_accuracy
                best_accuracy_accuracies_history = curr_accuracies_history
                best_accuracy_losses_history = curr_losses_history
                    
            if curr_EMA_accuracy > best_EMA_accuracy:
                best_EMA_accuracy_model = curr_EMA_accuracy_model
                best_EMA_accuracy = curr_EMA_accuracy
                best_EMA_accuracy_accuracies_history = curr_accuracies_history
                best_EMA_accuracy_losses_history = curr_losses_history
        
        #after final all repeats of current experiment
        
        #global save best of experiment results
        
        #####for best accuracy
        output_of_experiments[-1]['best_accuracy'] = best_accuracy
        output_of_experiments[-1]['best_accuracy_accuracies_history'] = best_accuracy_accuracies_history
        output_of_experiments[-1]['best_accuracy_losses_history'] = best_accuracy_losses_history
        #get predictions (probabilities) of model
        df_predictions = get_test_predictions(best_accuracy_model, 
                                              images_after_rembg = experiment_params['images_after_rembg'], 
                                              batch_size = experiment_params['batch_size'])
        
        output_of_experiments[-1]['best_accuracy_predictions'] = df_predictions.copy()
        
        #get class predictions of model
        #threshold (отсечка) для отнесения к классу
        threshold = output_of_experiments[-1]['best_accuracy_predictions'].label.quantile(quantile_threshold)
        df_predictions['label'] = df_predictions['label'].map(lambda pred: 'dirty' if pred >= threshold else 'cleaned') 
        output_of_experiments[-1]['best_accuracy_threshold'] = threshold
        output_of_experiments[-1]['best_accuracy_class_predictions'] = df_predictions
        
        #save model
        output_of_experiments[-1]['best_accuracy_model'] = copy.deepcopy(best_accuracy_model)
        
        
        #####for best EMA accuracy
        output_of_experiments[-1]['best_EMA_accuracy'] = best_EMA_accuracy
        output_of_experiments[-1]['best_EMA_accuracy_accuracies_history'] = best_EMA_accuracy_accuracies_history
        output_of_experiments[-1]['best_EMA_accuracy_losses_history'] = best_EMA_accuracy_losses_history
        #get predictions (probabilities) of model
        df_predictions = get_test_predictions(best_EMA_accuracy_model, 
                                              images_after_rembg = experiment_params['images_after_rembg'], 
                                              batch_size = experiment_params['batch_size'])
        output_of_experiments[-1]['best_EMA_accuracy_predictions'] = df_predictions.copy()
        
        #get class predictions of model
        #threshold (отсечка) для отнесения к классу
        threshold = output_of_experiments[-1]['best_EMA_accuracy_predictions'].label.quantile(quantile_threshold)
        df_predictions['label'] = df_predictions['label'].map(lambda pred: 'dirty' if pred >= threshold else 'cleaned') 
        output_of_experiments[-1]['best_EMA_accuracy_threshold'] = threshold
        output_of_experiments[-1]['best_EMA_accuracy_class_predictions'] = df_predictions
        
        #save model
        output_of_experiments[-1]['best_EMA_accuracy_model'] = copy.deepcopy(best_EMA_accuracy_model)
        
        
        
        #check/save global best models
        if best_accuracy > global_best_accuracy:
            global_best_accuracy_model = best_accuracy_model
            global_best_accuracy = best_accuracy
            global_best_accuracy_index = experiment_index
                    
        if best_EMA_accuracy > global_best_EMA_accuracy:
            global_best_EMA_accuracy_model = best_EMA_accuracy_model
            global_best_EMA_accuracy = best_EMA_accuracy
            global_best_EMA_accuracy_index = experiment_index

    #after final all experiments
    
    print("#########################################################################################")
    print("##############################Complete all experiments###################################")
    print("Best accuracy = {:.3f}, in experiment name = {}".format(global_best_accuracy, 
                                                               output_of_experiments[global_best_accuracy_index]['experiment_name']))
    print("Best EMA accuracy = {:.3f}, in experiment name = {}".format(global_best_EMA_accuracy, 
                                                               output_of_experiments[global_best_EMA_accuracy_index]['experiment_name']))
    print("#########################################################################################")
    
    return global_best_accuracy_model, global_best_EMA_accuracy_model, global_best_accuracy_index, global_best_EMA_accuracy_index, output_of_experiments
            

In [32]:
global_best_accuracy_model, global_best_EMA_accuracy_model, global_best_accuracy_index, \
global_best_EMA_accuracy_index, output_of_experiments = run_all_experiments(list_of_experiments_params)

############## Start experiment with name: Adam_default_params_no_rembg_drop_0.4 ##############
Epoch =   0/29,     ACCURACY: val = 0.648, train = 0.625, ema = 0.648,      LOSS: val = 2.212, train = 0.685
Epoch =   1/29,     ACCURACY: val = 0.352, train = 0.500, ema = 0.441,      LOSS: val = 2.823, train = 2.125
Epoch =   2/29,     ACCURACY: val = 0.359, train = 0.500, ema = 0.383,      LOSS: val = 1.550, train = 2.662
Epoch =   3/29,     ACCURACY: val = 0.652, train = 0.500, ema = 0.571,      LOSS: val = 0.989, train = 1.704
Epoch =   4/29,     ACCURACY: val = 0.648, train = 0.800, ema = 0.625,      LOSS: val = 2.204, train = 0.466
Epoch =   5/29,     ACCURACY: val = 0.648, train = 0.525, ema = 0.641,      LOSS: val = 1.566, train = 1.914
Epoch =   6/29,     ACCURACY: val = 0.849, train = 0.575, ema = 0.787,      LOSS: val = 0.327, train = 0.897
Epoch =   7/29,     ACCURACY: val = 0.702, train = 0.925, ema = 0.727,      LOSS: val = 0.643, train = 0.148
Epoch =   8/29,     ACCURACY: va

In [33]:
#######generate and save kernel outputs#######


#save submission of best accuracy model
index = global_best_accuracy_index
best_acc_type = 'best_accuracy'
output_of_experiments[index]['{}_class_predictions'.format(best_acc_type)].to_csv(
                'submission_{}_{}_{}_tr-{:.3f}.csv'.format(best_acc_type, 
                datetime.now().strftime("%Y-%m-%d_%H-%M-%S"),
                output_of_experiments[index]['experiment_name'], 
                output_of_experiments[index]['{}_threshold'.format(best_acc_type)]))

#save submission of best EMA accuracy model
index = global_best_EMA_accuracy_index
best_acc_type = 'best_EMA_accuracy'
output_of_experiments[index]['{}_class_predictions'.format(best_acc_type)].to_csv(
                'submission_{}_{}_{}_tr-{:.3f}.csv'.format(best_acc_type, 
                datetime.now().strftime("%Y-%m-%d_%H-%M-%S"),
                output_of_experiments[index]['experiment_name'], 
                output_of_experiments[index]['{}_threshold'.format(best_acc_type)]))

#generate/save submission of ansamble of best accuracy models
list_of_series = []
best_acc_type = 'best_accuracy'
for experiment in output_of_experiments:
    list_of_series.append(experiment['{}_class_predictions'.format(best_acc_type)]['label'].rename(experiment['experiment_name']))
df = pd.concat(list_of_series, axis=1, keys=[s.name for s in list_of_series])
df = df.replace({'dirty': 1, 'cleaned': 0}).mean(axis=1).rename('label').to_frame()

threshold = df.label.quantile(1.0 - 486/744)
df['label'] = df['label'].map(lambda pred: 'dirty' if pred >= threshold else 'cleaned')

df.to_csv('submission_ansamble_{}_{}_tr-{:.3f}.csv'.format(best_acc_type, 
                datetime.now().strftime("%Y-%m-%d_%H-%M-%S"), threshold))

#generate/save submission of ansamble of best EMA accuracy models
list_of_series = []
best_acc_type = 'best_EMA_accuracy'
for experiment in output_of_experiments:
    list_of_series.append(experiment['{}_class_predictions'.format(best_acc_type)]['label'].rename(experiment['experiment_name']))
df = pd.concat(list_of_series, axis=1, keys=[s.name for s in list_of_series])
df = df.replace({'dirty': 1, 'cleaned': 0}).mean(axis=1).rename('label').to_frame()

threshold = df.label.quantile(1.0 - 486/744)
df['label'] = df['label'].map(lambda pred: 'dirty' if pred >= threshold else 'cleaned')

df.to_csv('submission_ansamble_{}_{}_tr-{:.3f}.csv'.format(best_acc_type, 
                datetime.now().strftime("%Y-%m-%d_%H-%M-%S"), threshold))


####save full outputs of all experiments####
#https://stackoverflow.com/questions/4529815/saving-an-object-data-persistence/4529901
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as inp:
        return pickle.load(inp)

# sample usage
#company1 = [1,2,3,4,5]
#save_object(company1, '/kaggle/working/company1.pkl')
#del company
#company1 = load_object(filename)

save_object(output_of_experiments, '/kaggle/working/output_of_experiments_{}.pkl'.format(datetime.now().strftime("%Y-%m-%d_%H-%M-%S")))

In [34]:
#dir(models)

# # Disable grad for all conv layers (скорей всего морозим градиенты не только conv)
# for param in model.parameters():
#     param.requires_grad = False
    
# #заменяем последний полносвязный слой, содержащий 1000 нейронов (для 1000 классов imagenet1000)
# #на полносвязный слой из 2-х нейронов, для наших 2-х классов (кстати странно что не используется 
# #классическая бинарная конфигурация, с 1-м выходом и BCE, нужно проверить, даст ли замена на 
# #классику более лучший результат)
# #последний слой хранится в параметре .fc экземпляра класса, до замены он был = 
# #Linear(in_features=512, out_features=1000, bias=True)
# if dropout_p > 0:
#     model.fc = torch.nn.Sequential(torch.nn.Dropout(dropout_p), torch.nn.Linear(model.fc.in_features, 2))
# else:
#     model.fc = torch.nn.Linear(model.fc.in_features, 2)

# model = model.to(device)

# loss = torch.nn.CrossEntropyLoss()

# if optimizer_type == 'Adam':
#     optimizer = torch.optim.Adam(model.parameters(), lr=optimizer_Adam_lr, weight_decay=optimizer_weight_decay, amsgrad = optimizer_amsgrad)

# elif optimizer_type == 'SGD':
#     optimizer = torch.optim.SGD(model.parameters(), lr=optimizer_SGD_lr, momentum=optimizer_SGD_momentum, weight_decay=optimizer_weight_decay)

# # Decay LR by a factor of 0.1 every 7 epochs
# # Единственное назначение .lr_scheduler.StepLR, это изменение LR, оптимизатора, путем его домножения 
# # на gamma, каждые step_size эпох
# # Важно понимать, что scheduler считает эпохи путем явного выхова scheduler.step(),
# # а так же, что его .step() не делает ничего более кроме изменения LR каждые step_size эпох т.е. 
# # scheduler.step() не отменяет необходимости вызова optimizer.step() для осуществления град. шага

# # gamma - гиперпараметр, соответсвенно имеет смысл им поиграть на практике и найти оптимальный
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step_size, gamma=scheduler_gamma)

In [35]:
# experiment_name = 'resnet152_random_changes'
# best_accuracy_model, best_EMA_accuracy_model, best_accuracy, best_EMA_accuracy, accuracies[experiment_name], losses[experiment_name] = \
# train_model(model, loss, optimizer, scheduler, num_epochs=num_epochs, with_validation = with_validation, experiment_name = experiment_name)

In [36]:
# show_experiments_plots(accuracies, losses)

In [37]:
# threshold = submission_df.label.quantile(1.0 - 486/744)
# submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred >= threshold else 'cleaned') #threshold (отсечка) для отнесения к классу 

In [38]:
# test_dataset = ImageFolderWithPaths('/kaggle/working/test', val_transforms)

# test_dataloader = torch.utils.data.DataLoader(
#     test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [39]:
# #model_last
# model.eval() #не забываем перевести модель в eval режим

# test_predictions = []
# test_img_paths = []
# for inputs, labels, paths in iter(test_dataloader):
#     inputs = inputs.to(device)
#     labels = labels.to(device)
#     with torch.set_grad_enabled(False):
#         preds = model(inputs)
#     test_predictions.append(
#         torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
#     test_img_paths.extend(paths)
    
# test_predictions = np.concatenate(test_predictions) #преобразуем list в np.array

In [40]:
# iter_test_dataloader = iter(test_dataloader)
# inputs_slice = - batch_size

In [41]:
# inputs_slice += batch_size
# inputs, labels, paths = next(iter_test_dataloader)

# for img, pred in zip(inputs, test_predictions[inputs_slice:]):
#     show_input(img, title=pred)

In [42]:
# submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})
# threshold = submission_df.label.quantile(1.0 - 486/744)
# submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred >= threshold else 'cleaned') #threshold (отсечка) для отнесения к классу 
# submission_df['id'] = submission_df['id'].str.replace('/kaggle/working/test/unknown/', '')
# submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
# submission_df['id'] = submission_df['id'].str.replace('.png', '')
# submission_df.set_index('id', inplace=True)
# #submission_df.head(n=6)

In [43]:
# submission_df.to_csv('submission_last_model_{}_{}_tr-{:.3f}.csv'.format(datetime.now().strftime("%Y-%m-%d_%H-%M-%S"),
#                                                   experiment_name, threshold))

In [44]:
# if with_validation:
#     best_acc_type = 'val'
# else:
#     best_acc_type = 'train'

In [45]:
!rm -rf train val test plates plates_rbg __MACOSX

In [46]:
!ls

__notebook__.ipynb
output_of_experiments_2022-01-09_14-58-24.pkl
submission_ansamble_best_EMA_accuracy_2022-01-09_14-58-24_tr-0.500.csv
submission_ansamble_best_accuracy_2022-01-09_14-58-24_tr-0.500.csv
submission_best_EMA_accuracy_2022-01-09_14-58-24_Adam_default_params_autoaug_no_rembg_drop_0.4_tr-0.449.csv
submission_best_accuracy_2022-01-09_14-58-24_SGD_default_params_lr_0.01_mom_0.9_cjitter_drop_0.6_tr-0.503.csv
